In [ ]:
!pip install mlxtend
!pip install pandas

In [106]:
import time
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules


## Read data from csv file

In [107]:

with open("supermarket.csv") as f:
    content = f.readlines()

transaction_list = []

for line in content: 
    # print(line)
    items_purchased = [i.strip() for i in line.split(',')]
    # print(items_purchased)
    transaction_list.append(items_purchased)

# print(transaction_list[:5])  # Sanity check of the result

## Preprocess transaction to one-hot encoding

In [108]:
te = TransactionEncoder()
te_transactions = te.fit(transaction_list).transform(transaction_list)
df = pd.DataFrame(te_transactions, columns=te.columns_)
print(len(te.columns_))
# print(df)

124


## Timer Wrapper to collect function running time 

In [109]:
def timer(fun):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = fun(*args, **kwargs)
        print("--- %s seconds ---" % (round(time.time() - start_time, 3)))
        return res
    return wrapper

## Binary search function for the `fp_growth` algorithm 

In [135]:
@timer
def search_fp_growth(data_frame, itemsets_needed, treshold=0.01, debug=False):
    hi = 1
    lo = 0
    final_min_tresh = lo
    cnt = 0
    itemsets = []
    while((hi-lo)>treshold):
        cnt +=1
        mid = (lo+hi)/2
        itemsets = fpgrowth(df=data_frame, min_support=mid, use_colnames=True)
        if len(itemsets)<itemsets_needed:
            hi=mid
        else:
            final_min_tresh=max(final_min_tresh, mid)
            lo=mid
    print("Number of iterations: ",cnt)
    print("Final min support=",final_min_tresh)
    itemsets = fpgrowth(df=data_frame, min_support=final_min_tresh, use_colnames=True)
    return itemsets

## Get the itemsets

In [136]:
wanted_itemset_length = 50
print(wanted_itemset_length)
final_itemsets = search_fp_growth(df, itemsets_needed=wanted_itemset_length, treshold=0.001)
final_itemsets.sort_values('support', ascending=False)

50
Number of iterations:  10
Final min support= 0.365234375
--- 2.052 seconds ---


,support,itemsets
0,0.719689,(bread and cake)
1,0.640156,(fruit)
2,0.639939,(vegetables)
14,0.637130,(total = low)
3,0.635185,(milk-cream)
4,0.604063,(baking needs)
5,0.587206,(frozen foods)
6,0.563000,(biscuits)
7,0.532310,(juice-sat-cord-ms)
24,0.505079,"(bread and cake, milk-cream)"


## Binary search function for the association rules generation

In [127]:
@timer
def search_association_rules(frequent_itemsets, rules_needed, arguments, treshold=0.01, debug = False):
    hi = 100
    lo = arguments['min_threshold']
    rules = []
    final_min_tresh = lo
    while((hi-lo)>treshold):
        mid = (lo+hi)/2
        arguments['min_threshold'] = mid
        rules = association_rules(frequent_itemsets, **arguments)
        if debug:
            print("lo=",lo,"mid=",mid,"hi=",hi,arguments, "len(rules)=",len(rules))
        if len(rules)<rules_needed:
            hi=mid
        else:
            final_min_tresh=max(final_min_tresh, mid)
            lo=mid
    print("Final min ",arguments['metric'],"=",final_min_tresh)
    arguments['min_threshold'] =final_min_tresh
    rules = association_rules(frequent_itemsets, **arguments)
    if debug:
        print(final_min_tresh,len(rules))
    return final_min_tresh, rules

In [130]:
association_rules_arguments = dict(
    metric="lift", 
    min_threshold=1
)
final_threshold, final_rules = search_association_rules(final_itemsets, 15*2, association_rules_arguments, treshold=0.001)
print(len(final_rules))
print(final_rules.sort_values('lift', ascending=False)[:][['antecedents', 'consequents', 'lift', 'confidence' ]])

Final min  lift = 1.0891265869140625
--- 0.063 seconds ---
30
                     antecedents                   consequents      lift  \
9                        (fruit)  (bread and cake, vegetables)  1.217475   
4   (bread and cake, vegetables)                       (fruit)  1.217475   
5        (bread and cake, fruit)                  (vegetables)  1.203743   
8                   (vegetables)       (bread and cake, fruit)  1.203743   
22                    (biscuits)                (frozen foods)  1.183261   
23                (frozen foods)                    (biscuits)  1.183261   
3                        (fruit)                  (vegetables)  1.164336   
2                   (vegetables)                       (fruit)  1.164336   
6            (vegetables, fruit)              (bread and cake)  1.127583   
7               (bread and cake)           (vegetables, fruit)  1.127583   
25                    (biscuits)                (baking needs)  1.121008   
24                (baking 

In [114]:
@timer
def start_association_rules(frequent_itemsets, arguments):
    rules = association_rules(frequent_itemsets, **arguments)
    return rules

## Verify binary search result 

In [115]:
association_rules_arguments = dict(
    metric="lift", 
    min_threshold=final_threshold
)
rules = start_association_rules(final_itemsets, association_rules_arguments)
print(len(rules))
print(rules.sort_values('lift', ascending=False)[:][['antecedents', 'consequents', 'lift', 'confidence' ]])

--- 0.004 seconds ---
20
                     antecedents                   consequents      lift  \
2   (bread and cake, vegetables)                       (fruit)  1.217475   
7                        (fruit)  (bread and cake, vegetables)  1.217475   
3        (bread and cake, fruit)                  (vegetables)  1.203743   
6                   (vegetables)       (bread and cake, fruit)  1.203743   
14                    (biscuits)                (frozen foods)  1.183261   
15                (frozen foods)                    (biscuits)  1.183261   
0                   (vegetables)                       (fruit)  1.164336   
1                        (fruit)                  (vegetables)  1.164336   
4            (vegetables, fruit)              (bread and cake)  1.127583   
5               (bread and cake)           (vegetables, fruit)  1.127583   
17                    (biscuits)                (baking needs)  1.121008   
16                (baking needs)                    (biscuits) 